* https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial
* https://query.wikidata.org/
* https://janakiev.com/blog/wikidata-mayors/

In [ ]:
import requests
import pandas as pd
from collections import OrderedDict

In [ ]:
def get_voivodeships_v1():
    # problem - only ten voivodeships has the inception date se
    # solution - version 2 of this function
    
    # request voivodeships of poland(wd:Q36)
    # https://query.wikidata.org/#%20%20%20%20SELECT%20%3Fvoivodeship%20%3FvoivodeshipLabel%20%3Finception%20%3Flocation%0A%20%20%20%20WHERE%20%7B%0A%20%20%20%20%20%20%3Fvoivodeship%20wdt%3AP31%20wd%3AQ150093%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP571%20%3Finception%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP625%20%3Flocation.%0A%20%20%20%20%20%20FILTER%28%3Finception%20%3E%3D%20%221999-01-01%22%5E%5Exsd%3AdateTime%29.%0A%0A%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%20%20%20%20%7D
    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?voivodeship ?voivodeshipLabel ?inception ?location
    WHERE {
      ?voivodeship wdt:P31 wd:Q150093;
                   wdt:P571 ?inception;
                   wdt:P625 ?location.
      FILTER(?inception >= "1999-01-01"^^xsd:dateTime).

      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    # convert json to dataframe
    voivodeships = []
    for item in data['results']['bindings']:
        voivodeships.append(OrderedDict(
        {
            'voivodeship':      item['voivodeshipLabel']['value'],
            'location':         item['location']['value'],        
            'wikidata_item_id': item['voivodeship']['value'].split('/')[-1]
        }))
    #     print(item,'\n')

    df = pd.DataFrame(voivodeships)
    return df

get_voivodeships_v1()

,voivodeship,location,wikidata_item_id
0,Lower Silesian Voivodeship,Point(16.383333333 51.083333333),Q54150
1,Kuyavian-Pomeranian Voivodeship,Point(18.483333333 53.066666666),Q54153
2,Lublin Voivodeship,Point(22.9 51.216666666),Q54155
3,Lesser Poland Voivodeship,Point(20.266666666 49.85),Q54159
4,Masovian Voivodeship,Point(21.1 52.333333333),Q54169
5,Opole Voivodeship,Point(17.9 50.633333333),Q54171
6,Podkarpackie Voivodeship,Point(22.166666666 49.95),Q54175
7,Podlaskie Voivodeship,Point(22.931938888 53.267219444),Q54177
8,Silesian Voivodeship,Point(19.0 50.333333333),Q54181
9,Greater Poland Voivodeship,Point(17.233333333 52.333333333),Q54187


In [ ]:
def get_voivodeships_v2():
    # problem - too many voivodeships, historical ones are mixed together
    # solution - just cut the list after 16-th item
    
    # request voivodeships of poland(wd:Q36)
    # cut after 16 voivodeships, the rest are historical ones
    # https://query.wikidata.org/#SELECT%20%3Fvoivodeship%20%3FvoivodeshipLabel%20%3Flatitude%20%3Flongitude%20%3Fadmininistrative_teritorial_entity%0AWHERE%20%7B%0A%20%20%3Fvoivodeship%20wdt%3AP31%20wd%3AQ150093%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP131%20%3Fadmininistrative_teritorial_entity%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20p%3AP625%2Fpsv%3AP625%20%5B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLatitude%20%3Flatitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AgeoLongitude%20%3Flongitude%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%5D.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20FILTER%28%3Fadmininistrative_teritorial_entity%20%3D%20wd%3AQ36%29.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22pl%22.%20%7D%0A%7D%0AORDER%20BY%20DESC%28%3Fvoivodeship%29%0ALIMIT%2016
    # https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Mountains
    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?voivodeship ?voivodeshipLabel ?latitude ?longitude ?admininistrative_teritorial_entity
    WHERE {
      ?voivodeship wdt:P31 wd:Q150093;
                   wdt:P131 ?admininistrative_teritorial_entity;
                   p:P625/psv:P625 [
                       wikibase:geoLatitude ?latitude ;
                       wikibase:geoLongitude ?longitude ;
                   ].               
      FILTER(?admininistrative_teritorial_entity = wd:Q36).
      SERVICE wikibase:label { bd:serviceParam wikibase:language "pl". }
    }
    ORDER BY DESC(?voivodeship)
    LIMIT 16
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    # convert json to dataframe
    voivodeships = []
    for item in data['results']['bindings']:
        voivodeships.append(OrderedDict(
        {
            'voivodeship':      item['voivodeshipLabel']['value'],
            'latitude':         float(item['latitude']['value']),        
            'longitude':         float(item['longitude']['value']),                    
            'wikidata_item_id': item['voivodeship']['value'].split('/')[-1]
        }))
    #     print(item,'\n')    
    
    df = pd.DataFrame(voivodeships)
    return df

get_voivodeships_v2()

,voivodeship,latitude,longitude,wikidata_item_id
0,województwo zachodniopomorskie,53.600000,15.533333,Q54188
1,województwo wielkopolskie,52.333333,17.233333,Q54187
2,województwo warmińsko-mazurskie,53.850000,20.816667,Q54184
3,województwo świętokrzyskie,50.750000,20.766667,Q54183
4,województwo śląskie,50.333333,19.000000,Q54181
5,województwo pomorskie,54.183333,18.000000,Q54180
6,województwo podlaskie,53.267219,22.931939,Q54177
7,województwo podkarpackie,49.950000,22.166667,Q54175
8,województwo opolskie,50.633333,17.900000,Q54171
9,województwo mazowieckie,52.333333,21.100000,Q54169


In [ ]:
def coordinates_of_voivodeship(wojewodztwo):
    voivodeships = get_voivodeships_v2()
    coordinates = voivodeships.loc[voivodeships['voivodeship'].str.contains(wojewodztwo) == True, ['latitude', 'longitude']]
    return coordinates.values.tolist()[0]

In [ ]:
coordinates_of_voivodeship('województwo podlaskie')

[53.267219444444, 22.931938888889]

In [ ]:
coordinates_of_voivodeship('podlaskie')

[53.267219444444, 22.931938888889]

trzeba to rozwinąć o nazwy zgodne z modelem, czyli kolejno: powiaty, miasta i dzielnice- zrobisz to?